In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os 
import shutil 
import glob
import numpy as np
import matplotlib.pyplot as plt
import pygrib
import pickle
import datetime
import pandas as pd
from util import boxbin 

#################################################################
# Default plotting parameters
FIGURESIZE=(10,6)
FONTSIZE=  18
plt.rcParams['figure.figsize'] = FIGURESIZE
plt.rcParams['font.size'] = FONTSIZE
plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE
#################################################################

In [3]:


#load the gfs grid
static_inputs = pickle.load(open('./Archive/static_inputs.pkl','rb'))
lat = static_inputs['lat']#2D
lon = static_inputs['lon']#2D


#add the next grid point so the binning algorithm 
#size is 256x128
bin_lat = np.concatenate([lat[:,0],[53.25]])
bin_lon = np.concatenate([lon[0,:],[298]])

#store the sorted grid
xedge = np.sort(bin_lon)
yedge = np.sort(bin_lat)

xmid = [] #Blank array
ymid = [] #Blank array

#calcuate the midpoints for the sorting algorithm
i=0
while(i < len(xedge)-1):
    xmid.append((xedge[i]+xedge[i+1])/2) #Calculate and append midpoints
    i+=1 
i=0
while(i < len(yedge)-1):
    ymid.append((yedge[i]+yedge[i+1])/2) #Calculate and append midpoints
    i+=1

NameError: name 'sat' is not defined

In [15]:
#set the satellite and year
sat = 'G16'
yr = '2019'
hr = '12'



#declare the coordinates you need from the glm data raw files
coords = ['flash_lat','flash_lon','flash_id','flash_time_offset_of_first_event']
ltg_vars = ['flash_area','flash_energy','flash_quality_flag']
df_cols = np.concatenate([coords,ltg_vars])

save_dir = '/ourdisk/hpc/ai2es/bmac87/OG_datasets/GLM/'+sat+'/flashes_1hr_df/'
hours = ['00','01','02','03','04','05','06','07','08','09','10','11','12',
        '13','14','15','16','17','18','19','20','21','22','23']

for yr in ['2019','2020','2021','2022','2023','2024']: 
    #build the array of julian days. add 366 for leap year
    j_days = []
    for i in range(1,366):
        j_days.append(f"{i:03}")
    if yr=='2020':
        j_days.append('366')

    #loop through the files
    for d,day in enumerate(j_days):
        if d>=1:
            og_nc_dir = '/ourdisk/hpc/ai2es/bmac87/OG_datasets/GLM/'+sat+'/'+yr+'/'+day+'/'
            og_nc_files = sorted(os.listdir(og_nc_dir))
            for h,hr in enumerate(hours):
                print(yr,day,hr)
                if h>=0:

                    #OR_GLM-L2-LCFA_G16_s20190010000000_e20190010000200_c20190010000227.nc
                    hr_files = sorted(glob.glob(og_nc_dir+'OR_GLM-L2-LCFA_'+sat+'_s'+yr+day+hr+'*.nc'))
                    for f,file in enumerate(hr_files):
                        try:
                            ds = xr.open_dataset(file)
                            ds = ds[ltg_vars] #get the flash information using the dataset variables
                            df = ds.to_dataframe() #convert to dataframe
                            df.index = df['flash_time_offset_of_first_event']#set the indices
                            df = df[df_cols] #subset the flash information with Lat/Lon
                            
                            if f==0:
                                flashes_df = df
                            else:
                                flashes_df = pd.concat([flashes_df,df])
                        #end try

                        except FileNotFoundError:
                            print(file, 'file not found')
                        #end_except

                    #end_for_files
                    if not os.path.isdir(save_dir):
                        os.mkdirs(save_dir)
                    pickle.dump(flashes_df,open(save_dir+sat+'_'+yr+'_'+day+'_'+hr+'.pkl','wb'))
                    del flashes_df, df, ds
                #end_hours_test
            #end_for_hours
        #end_days_test
    #end_for_days
#end_for_yrs



       

2019 002 00
2019 002 01
2019 002 02
2019 002 03
2019 002 04
2019 002 05


KeyboardInterrupt: 

## Test pickle output

In [13]:
hr = '15'
day ='001'
yr='2019'

df = pickle.load(open(save_dir+sat+'_'+yr+'_'+day+'_'+hr+'.pkl','rb'))
df

,flash_lat,flash_lon,flash_id,flash_time_offset_of_first_event,flash_area,flash_energy,flash_quality_flag
flash_time_offset_of_first_event,,,,,,,
2019-01-01 15:00:00.382239341,-36.582981,-60.864941,21511,2019-01-01 15:00:00.382239341,67.907829,3.204537e-14,0.0
2019-01-01 15:00:00.513085365,-38.438507,-63.527039,21512,2019-01-01 15:00:00.513085365,69.739052,9.155820e-15,0.0
2019-01-01 15:00:00.122454643,-32.107563,-57.497242,21510,2019-01-01 15:00:00.122454643,975.736328,8.499653e-13,0.0
2019-01-01 15:00:00.743115425,-30.705791,-59.289330,21513,2019-01-01 15:00:00.743115425,73.248894,1.068179e-14,0.0
2019-01-01 15:00:01.204319477,-29.417583,-57.866787,21514,2019-01-01 15:00:01.204319477,290.096161,1.419152e-13,0.0
...,...,...,...,...,...,...,...
2019-01-01 15:59:58.806747436,-31.078432,-58.951519,44724,2019-01-01 15:59:58.806747436,493.972260,7.477253e-14,0.0
2019-01-01 15:59:59.057758331,-38.623978,-62.729149,44725,2019-01-01 15:59:59.057758331,140.241119,1.068179e-14,0.0
2019-01-01 15:59:59.202339172,-18.332787,-57.272587,44728,2019-01-01 15:59:59.202339172,149.397232,2.441552e-14,0.0


In [ ]:
 # print(og_nc_files[0])
        # print(og_nc_files[-1])
        # for nc_file in og_nc_files:
        #     print(nc_file)



    # try:
    #     ds = xr.open_dataset(glm_dir+glm_file,engine='netcdf4')
    #     df = ds.to_dataframe()
    # except FileNotFoundError:
    #     print('no file',glm_dir+glm_file)
    #     continue
        
    #     df['date_time'] = df.index
    #     days = df['date_time'].dt.day.drop_duplicates().values
    #     df_last_day = df.loc[((df['date_time'].dt.day==days[0]))]

    #     hours = np.linspace(0,23,24)
    #     for hour in hours:
#             if hour==0:
#                 df_1hr = pd.concat([df_last_day,df.loc[((df['date_time'].dt.hour==hour))]])
#             elif hour==23:
#                 df_1hr = df.loc[((df['date_time'].dt.day==days[1]))]
#                 df_1hr = df_1hr.loc[(df_1hr['date_time'].dt.hour==hour)]
#             else:
#                 df_1hr = df.loc[((df['date_time'].dt.hour==hour))]
# df_1hr